<div align="right" vertical-align="middle" style="border: 2px solid;border-radius: 5px;background-color:lightgrey;padding:5px;padding-right:20px;padding-left:10px;">
        <a style="color:black;text-decoration:none;" title="Home" href="../index.ipynb">
            <img src="../../css/iconmonstr-christmas-house-icon.svg" height = "30" width = "30" style="display:inline">
        </a>
        &nbsp;
        <b>|</b>
        &nbsp;
        <a style="color:black;text-decoration:none;" title="Build" href="../build_docs/build.ipynb">
            <img src="../../css/iconmonstr-puzzle-icon.svg" height = "30" width = "30" style="display:inline">
        </a>
        <a style="color:black;text-decoration:none;" title="Assemble" href="assemble.ipynb">
            <img src="../../css/iconmonstr-puzzle-17-icon.svg" height = "30" width = "30" style="display:inline">
        </a>
        <a style="color:black;text-decoration:none;" title="Query" href="../query_docs/query.ipynb">
            <img src="../../css/iconmonstr-flask-3-icon.svg" height = "30" width = "30" style="display:inline">
        </a>
</div>

# TUTORIAL: Assemble an EGRIN 2.0 Ensemble

####*Important!!! This tutorial assumes you have access to a complete cMonkey2 ensemble.*

### In a nutshell

The `ASSEMBLE` scripts transfers and compiles individual [cMonkey2](https://github.com/baliga-lab/cmonkey2) SQLite databases into an integrated MongoDB database. 

In addition, they perform several post-processing steps, including: detection of gene regulatory elements (GREs) by comparing individual bicluster motifs with [`TOMTOM`](http://meme.ebi.edu.au/meme/doc/tomtom.html) and clustering with [MCL](http://micans.org/mcl/), genome-wide scanning of motifs with FIMO, and detection of co-regulated modules or **corems** using link-community detection.

### Requirements

User must supply several files, including:

- `ratios`: a Tab-delimited or Comma-separated file containing a matrix of gene expression values across all conditions.
- `blocks`: a Comma-separated file containing annotations for each condition.
- `inclusion_blocks`: : a Comma-separated file containing groups of blocks to be co-included in runs.
- `exclusion_blocks`: : a Comma-separated file containing groups of blocks to be co-excluded in runs.

Optionally:
- `pipeline`: a JSON file containing custom scoring pipeline. Currently only set-enrichment pipeline is supported
- `setenrich_files`: Comma-separated files containing set-enrichment sets. Multiple files should be separated by a comma.

The format for each of these files will be described in detail below.

Additionally, the Python modules described on the [Home page](../index.ipynb) are required to run these scripts.

### Scripts

`cMonkeyQSub.py`: The control function for `BUILD` scripts. Writes QSub script.